# Pandas

Let's go back to the very first problem from this morning. We have an incomplete record of Newark temperatures since 1893.

In [ ]:
import numpy
temperatures = numpy.loadtxt("data/newark-temperature-avg.txt")
temperatures

But instead of analyzing it with raw Numpy, let's use Pandas.

In [ ]:
import pandas
temperatures = pandas.Series(temperatures)
temperatures

Numpy was designed to do fast calculations with minimal dependencies.

Pandas was designed to make a data analyst's life easier.

In [ ]:
%matplotlib inline

In [ ]:
temperatures.plot()

Behold the turning of the seasons!

You can think of a Pandas Series as a Numpy array with bells and whistles, but it's more than that.

It is an _indexed_ Numpy array with bells and whistles.

In [ ]:
temperatures.index

In [ ]:
temperatures.index = numpy.loadtxt("data/newark-days-ago.txt")
temperatures.index.name = "days ago"

In [ ]:
temperatures.index

In [ ]:
temperatures.plot()

In [ ]:
temperatures.index = pandas.to_datetime(temperatures.index, unit="D", origin=pandas.Timestamp("2018-11-04"))
temperatures.index.name = "date"
temperatures.index

In [ ]:
temperatures.plot()

Now let's return to the problem of imputing the missing temperature data.

In [ ]:
min_temperatures = pandas.Series(numpy.loadtxt("data/newark-temperature-min.txt"))
max_temperatures = pandas.Series(numpy.loadtxt("data/newark-temperature-max.txt"))
min_temperatures.index = temperatures.index
max_temperatures.index = temperatures.index

In [ ]:
min_temperatures.plot()

In [ ]:
max_temperatures.plot()

It would be more convenient if these were in the same object. A DataFrame is several Series glued together with a common index.

In [ ]:
df = pandas.concat([temperatures, min_temperatures, max_temperatures], axis="columns")
df.columns = ["avg", "min", "max"]   # name them!
df

Performing calculations on columns of a DataFrame is very much like Numpy: all of the elementwise operations, masking, and fancy indexing apply. In fact, Pandas usually just passes off this work on Numpy. However, it _organizes_ that work.

In [ ]:
df["min-max avg"] = (df["min"] + df["max"]) / 2
df

Although we could have repeated the Numpy method of masking by `numpy.isnan(df["avg"])`, Pandas has functions for dealing with missing data. (A lot of them, actually.)

In [ ]:
df["imputed"] = df["avg"].fillna(df["min-max avg"])
df

We can select columns by strings in square brackets (like Numpy's record arrays, a feature I didn't show you), but rows cannot be selected by integer index.

Whereas an array of length `N` is a function `[0, N) → V`, a DataFrame is a function `K → V1×V2×V3`, where `K` is the index and `V1×V2×V3` are the columns. Integer indexing won't work unless the index has integer type:

In [ ]:
try:
    df[-1]
except KeyError:
    print("Nope, -1 is not a valid index for this DataFrame.")

But unlike an SQL table, a DataFrame index has an ordering. We should be able to ask for the last element, so there's a special accessor for that: `iloc`.

In [ ]:
df.iloc[-1]

But generally, the index type should be more meaningful than the positions, so we'd want to access rows by index elements. That's `loc`.

In [ ]:
df.loc[pandas.Timestamp("2018-11-04")]

We can slice regions of time just as we slice integer indexes.

In [ ]:
df.loc[pandas.Timestamp("2017-11-04"):].plot()

And that lets us zoom into interesting regions in the data space.

In [ ]:
df[["imputed", "avg"]].loc[pandas.Timestamp("2011-01-01"):pandas.Timestamp("2015-01-01")].plot()

Pandas has oodles of statistical functions. Whenever I want to do something, I do a web search to find out what it's named— _somebody's_ asked about it on StackOverflow.

Let's look for local warming! (Not global; this is just Newark.)

In [ ]:
df["imputed"].plot()

Day-to-day variations are drowning out any effect. Can we smooth these data?

Yes. It's called a rolling mean.

In [ ]:
df["imputed"].rolling(3*365).mean().plot()

The input files we've been working with are artificial (one number per line as text). The original file that came from the NCDC was a CSV with redundant information.

In [ ]:
df2 = pandas.read_csv("data/newark-temperature.csv")
df2

The first two columns have only one value because this was a database selection. Pandas's `read_csv` function recognized that the temperature columns are numbers, but not that the date is a date.

In [ ]:
df2["DATE"].dtype

The `'O'` type (object) is for cases when Pandas doesn't recognize the type of a field. In principle, it could be any Python object, but from a CSV file, it's a string.

We can try to read it again, this time providing a hint that this column is supposed to be a date.

In [ ]:
df2 = pandas.read_csv("data/newark-temperature.csv", parse_dates=["DATE"])
df2.columns

It has the same columns, but now the date is a date.

In [ ]:
df2["DATE"].dtype

We'd like this date column to be the index of the whole table, so we say so.

In [ ]:
df2.index = df2["DATE"]

Now we can do the same analysis we did before, but directly on the DataFrame from the CSV file, not by gluing together Series derived from Numpy arrays.

In [ ]:
df2["imputed"] = df2["TAVG"].fillna((df2["TMIN"] + df2["TMAX"]) / 2)
df2["imputed"].rolling(3*365).mean().plot()

Temperature data are fairly simple: a one-dimensional time series. Let's consider something with a bit more structure— exoplanets.

Each star may have a different number of planets, which complicates the indexing.

In [ ]:
planets = pandas.read_csv("data/nasa-exoplanets.csv")
planets

Not knowing much about this dataset, let's get a quick summary of the columns.

In [ ]:
planets.describe().transpose()

Fortunately, NASA gave us longer explanations of the column names.

In [ ]:
!cat data/nasa-exoplanets-details.txt

Number of moons? ("`pl_mnum`: Number of Moons in System") Awesome!

In [ ]:
planets[planets["pl_mnum"] > 0]

I guess not yet. They're just being hopeful.

In this dataset, one row is one planet. Thus, star data for stars with multiple known planets are duplicated (which effectively weights star data by their number of planets in `planets.describe()`).

To get a table of stars only, we'd have to do a group-by. We expect the star data to be the same for each planet associated with a star, so the mean is an appropriate summary.

In [ ]:
planets.groupby("pl_hostname").mean()

**Exercise:** This reduction averages all values per star, which is not meaningful for planet variables (which start with `"pl_"`). After all, what does it mean to average their "semi-major axes?" Or their "discovery methods?"

To avoid confusion, repeat the group-by with only the columns that start with `"st_"`.

In [ ]:
planets.groupby("pl_hostname")[[x for x in planets.columns if x.startswith("st_")]].mean()

Wouldn't it be great if the index encapsulated the hierarchical relationship between stars and planets?

Pandas has a `MultiIndex`, which allows sub-indexes to be nested within outer indexes.

In [ ]:
planets.index = pandas.MultiIndex.from_arrays([planets["pl_hostname"], planets["pl_letter"]])
planets

The transpose of this multiindexed table is a table with two levels of columns.

In [ ]:
planets.transpose()

To simplify this table, let's reduce it to the following fields:

  * `pl_discmethod`:  Discovery Method
  * `pl_orbper`:      Orbital Period [days]
  * `pl_orbsmax`:     Orbit Semi-Major Axis [AU])
  * `pl_orbeccen`:    Eccentricity
  * `pl_orbincl`:     Inclination [deg]
  * `pl_bmassj`:      Planet Mass or M*sin(i) [Jupiter mass]
  * `pl_bmassprov`:   Planet Mass or M*sin(i) Provenance
  * `pl_disc`:        Year of Discovery
  * `pl_telescope`:   Discovery Telescope

In [ ]:
simple = planets[["pl_discmethod", "pl_orbper", "pl_orbsmax", "pl_orbeccen", "pl_bmassj", "pl_bmassprov", "pl_disc", "pl_telescope"]]
simple

The syntax for selecting rows by star name now selects subtables of all planets associated with a star.

In [ ]:
simple.loc["tau Boo"]

In [ ]:
simple.loc["tau Cet"]

How about the opposite selection? To get all planets labeled `"b"`— this is one planet per star. The method for that is `xs` (for "cross-section").

In [ ]:
simple.xs("b", level="pl_letter")

In a sense, the multiindexed table represents three dimensional information: (1) star, (2) planet letter, (3) columns. Some combinations of star and planet letter do not exist: these two dimensions are not completely filled. Some stars have only one planet, while others have as many as...

In [ ]:
planets["pl_letter"].groupby("pl_hostname").count().max()

...8 planets. Most have only one. I would call this dimension "jagged" or "ragged." Pandas represents this _sparsely,_ by enumerating only the combinations that do exist.

Pandas used to have a type to handle 3+ dimensional data (Series is 1D, DataFrame is 2D, ...), but this was dropped in favor of multiindexes.

There's another library called xarray to fill this niche, but generally, I'd rather deal with 3+ dimensions sparsely than densely.

If we want to pass these data to a machine learning model or something, we'll have to flatten the star-planet structure.

A lossless way to do that is to pivot planet letter keys into columns. (Note: the `stack` and `unstack` functions do a similar thing.)

In [ ]:
simple.pivot_table(index="pl_hostname", columns="pl_letter")

There's now a column for each letter-column combination. Most of them are empty because there was one star with 8 planets but most have 1 planet.

If you want to simplify this table, you'll have to make choices because anything you do from here on loses information.

   * Do you fill in NaN with `fillna`?
   * Do you pick a few of the most common planet letters, like `["b", "c", "d"]`?
   * Do you average over all planets?
   * Do you turn the original into two tables, one for each star and another for each planet, with identifiers linking them?

**One last topic:** This dataset has a lot of strings. When we read it from the CSV file, Pandas left them as Python strings. This can be inefficient for very large tables (in storage and comparison speed). A common technique is to find unique strings and replace each value with an integer. This is exactly what we did with the Gettysburg Address this morning ("dictionary encoding"). Pandas has an automated way to do it.

In [ ]:
simple["pl_discmethod"].dtype

Type `'O'` is "object," for arbitrary Python objects (usually strings). We want it to be a "category" type. Let's convert it (`astype`) and assign it.

In [ ]:
simple["pl_discmethod"] = simple["pl_discmethod"].astype("category")

Uh oh... what's that warning?

It's one of the most common you get with Pandas ([here's a whole blog on it](https://www.dataquest.io/blog/settingwithcopywarning/)). Remember the "view vs copy" discussion this morning? This is the same thing in a Pandas, rather than Numpy, context.

Unlike Numpy, Pandas gives you a warning. We really should address that warning.

The issue was that `simple` is a view of `planets`, so modifying a column in `simple` modifies `planets`. That could lead to surprising results.

The solution? Turn `simple` into a copy.

In [ ]:
simple = simple.copy()

Now there's no warning because everything's fine.

In [ ]:
simple["pl_discmethod"] = simple["pl_discmethod"].astype("category")

In [ ]:
simple["pl_discmethod"].dtype

Now when we do analysis on `"pl_discmethod"`, it's number-crunching, rather than string-crunching.

In [ ]:
simple["pl_discmethod"].value_counts().plot.bar(logy=True)

Just poking around now... discovery method by date?

In [ ]:
simple.reset_index().groupby(["pl_disc", "pl_discmethod"]).count()["pl_letter"].unstack("pl_discmethod").fillna(0).plot()

Wow— a lot of transits! Why is that?

In [ ]:
planets[planets["pl_discmethod"] == "Transit"].groupby(["pl_disc", "pl_telescope"]).count()["pl_letter"].unstack("pl_telescope").fillna(0).plot(figsize=(10, 7))

Oh. Kepler. The first peak corresponds to the Kepler telescope's first mission, which ended early in instrument failure. The second mission, K2, worked around the failure to get results until earlier this year.

Other than transits, what are the most popular methods?

In [ ]:
simple[simple["pl_discmethod"] != "Transit"].reset_index().groupby(["pl_disc", "pl_discmethod"]).count()["pl_letter"].unstack("pl_discmethod").fillna(0).plot(figsize=(10, 7))

Pulsar timing, apparently.

On to Dask!